In [ ]:
#import the libraries
import pandas as pd
from tqdm import tqdm_notebook
from pprint import pprint
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [ ]:
#read in the csv file
#data = pd.read_csv('Consumer_Complaints.csv', nrows = 100000)
data = pd.read_csv('/content/main_data.csv',encoding='cp1252')

In [ ]:
#checking the first 5 roews of the data
data.head()

,Unnamed: 0,link,time,headline,content_part,Category,Source
0,0,https://timesofindia.indiatimes.com/india/lok-...,2020-09-14 09:55:20,Lok Sabha adjourned for an hour after offering...,Lok Sabha MPs paid tribute to former President...,India,TOI
1,1,https://timesofindia.indiatimes.com/india/up-b...,2020-09-14 09:36:34,UP BJP launches 'Seva Saptah' to mark PM Modi'...,For the week-long celebrations that will culmi...,India,TOI
2,2,https://timesofindia.indiatimes.com/india/pm-m...,2020-09-14 09:34:44,"PM Modi extends greetings on 'Hindi Diwas', co...",Prime Minister Narendra Modi on Monday extend...,India,TOI
3,3,https://timesofindia.indiatimes.com/india/pm-m...,2020-09-14 09:33:23,"PM Modi invites ideas, inputs from countrymen ...","""The greatest strength of #MannKiBaat is the d...",India,TOI
4,4,https://timesofindia.indiatimes.com/india/than...,2020-09-14 09:30:00,Thank all MPs for choosing path of duty even d...,"""The monsoon session begins today, there is co...",India,TOI


In [ ]:
#considering the 'consumer complaints' column
#data = data[['Consumer complaint narrative']]
data = data[['content_part']]

In [ ]:
data.head()

,content_part
0,Lok Sabha MPs paid tribute to former President...
1,For the week-long celebrations that will culmi...
2,Prime Minister Narendra Modi on Monday extend...
3,"""The greatest strength of #MannKiBaat is the d..."
4,"""The monsoon session begins today, there is co..."


In [ ]:
#drop the NA values if any
#data = data[['Consumer complaint narrative']].dropna()
data = data[['content_part']].dropna()


In [ ]:
#convert the data to lower case
#data = data['Consumer complaint narrative'].str.lower()
data = data['content_part'].str.lower()

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data.head()

,content_part
0,lok sabha mps paid tribute to former president...
1,for the week-long celebrations that will culmi...
2,prime minister narendra modi on monday extend...
3,"""the greatest strength of #mannkibaat is the d..."
4,"""the monsoon session begins today, there is co..."


### DATA PREPROCESSING

**For the preprocessing of the data we will be using the NLTK library. First the data have to converted into the tokens which can later be used to remove the stopwords, the puncutations, removing any alphanumeric characters and also converting all the tokens to their base form using lemmatization.**

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#tokenize the data into corresponding word tokens
#tokens_data = [word_tokenize(w) for w in data['Consumer complaint narrative'].values.tolist()]
tokens_data = [word_tokenize(w) for w in data['content_part'].values.tolist()]

In [ ]:
#explicitly specifying the strings that are not that useful in the dataset
#not_necessary = ['xxxx', 'xxxx xxxx xxxx', 'xxxx xxxx', 'xxxxxxxxxxxx', 'xxxx/xxxx/xxxx', 'xx/xx/xxxx']
not_necessary = ['\r\n\r\ni']

In [ ]:
#removing the above specified not that useful strings from our dataset
tokenized = []
for sent in tokens_data:
    tokens_new = []
    for token in sent:
        if token not in not_necessary:
            tokens_new.append(token)
    tokenized.append(tokens_new)

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#here we will be removing the stopwords, lemmatizing our tokens, getting rid of punctuations and finally getting a clean corpus
lemmatize = WordNetLemmatizer()
bow = []
word_tokens = []
for token in tqdm_notebook(tokenized):
    tokens = [w for w in token if w not in stopwords.words('english')]
    tokens = [w for w in tokens if w not in string.punctuation]
    tokens = [lemmatize.lemmatize(w) for w in tokens]
    tokens1 = [w for w in tokens if w.isalpha()]
    word_tokens.append(tokens1)
    tokens = ' '.join([w for w in tokens])
    bow.append(tokens)

In [ ]:
print(word_tokens[:2])

[['lok', 'sabha', 'mp', 'paid', 'tribute', 'former', 'president', 'pranab', 'mukherjee', 'legendary', 'indian', 'classical', 'vocalist', 'pandit', 'jasraj', 'chief', 'minister', 'ajit', 'jogi', 'madhya', 'pradesh', 'governor', 'lalji', 'tandon', 'uttar', 'pradesh', 'minister', 'kamal', 'rani', 'chetan', 'chauhan', 'minister', 'rahguvansh', 'prasad', 'singh', 'others', 'passed', 'away', 'recently'], ['celebration', 'culminate', 'september', 'party', 'sent', 'circular', 'state', 'unit', 'chief', 'suggesting', 'activity', 'could', 'taken', 'part', 'campaign']]


In [ ]:
#complaints = pd.DataFrame(bow, columns = ['COMPLAINTS'])
complaints = pd.DataFrame(bow, columns = ['content_part'])
complaints.head()

,content_part
0,lok sabha mp paid tribute former president pra...
1,week-long celebration culminate september 20 p...
2,prime minister narendra modi monday extended g...
3,`` greatest strength mannkibaat diverse range ...
4,`` monsoon session begin today corona responsi...


#### CREATING THE CORPUS

In [ ]:
from gensim.corpora import Dictionary

In [ ]:
#creating the dictionery
id2word = Dictionary(word_tokens)

In [ ]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in word_tokens]

In [ ]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]]


#### BUILDING THE TOPIC MODEL

In [ ]:
from gensim.models import ldamodel

In [ ]:
lda = ldamodel.LdaModel(corpus = corpus,
                        iterations = 100,
                        id2word = id2word,
                        num_topics = 20,
                        random_state = 101,
                        update_every = 1,
                        chunksize = 100,
                        passes = 20,
                        alpha = 'auto',
                        per_word_topics = True)

In [ ]:
#viewing the topics of the LDA model
pprint(lda.print_topics())

[(0,
  '0.031*"national" + 0.028*"lakh" + 0.026*"study" + 0.025*"test" + '
  '0.022*"told" + 0.018*"top" + 0.016*"student" + 0.015*"cricket" + '
  '0.014*"best" + 0.014*"medical"'),
 (1,
  '0.055*"last" + 0.030*"trump" + 0.029*"compensation" + 0.026*"five" + '
  '0.022*"end" + 0.022*"president" + 0.018*"mean" + 0.018*"council" + '
  '0.016*"announced" + 0.015*"south"'),
 (2,
  '0.040*"first" + 0.032*"open" + 0.026*"police" + 0.025*"meeting" + '
  '0.021*"win" + 0.021*"grand" + 0.019*"thiem" + 0.019*"slam" + '
  '0.015*"general" + 0.014*"title"'),
 (3,
  '0.050*"started" + 0.048*"report" + 0.033*"hospital" + 0.032*"asked" + '
  '0.032*"fresh" + 0.031*"board" + 0.024*"issue" + 0.021*"little" + '
  '0.020*"certain" + 0.018*"covid"'),
 (4,
  '0.047*"gst" + 0.032*"may" + 0.028*"say" + 0.023*"work" + 0.023*"many" + '
  '0.023*"virus" + 0.020*"around" + 0.018*"move" + 0.015*"past" + '
  '0.015*"revenue"'),
 (5,
  '0.029*"handloom" + 0.024*"market" + 0.022*"leader" + 0.021*"including" + '
  '0

***Topic 0 is a represented as---> ('0.216*"account" + 0.100*"card" + 0.051*"charge" + 0.031*"fee" + 0.028*"chase" + 0.026*"closed" + 0.020*"fraud" + 0.019*"charged" + 0.015*"opened" + 0.014*"transaction"')***

***It means the top 10 keywords that contribute to this topic are: ‘account’, ‘card’, ‘charge’, 'fee', 'closed'.... and so on and the weight of 'account’ on topic 0 is 0.216.***

***The weights that are assigned to each word reflect how important a keyword is to that topic. For example, for the Topic 0 the word 'account' has the highest weigth associated with it reflecting that it is the moset important word representing the Topic 0.***

In [ ]:
doc_lda = lda[corpus]

#### VISIALIZING THE TOPICS USING PyLDAVIS

In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 8.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=0cf0b888ab4029337fcce1eac0c3b30eb6a38cbcbbe50ff136e382572714227a
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=f71eae516c30b5c82101caa80a26c37912daf5a51ed8d920d523cae6448fc59e
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
import pyLDAvis
from pyLDAvis import gensim

In [ ]:



# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.284602 -0.007993       1        1  15.617618
16    -0.320639 -0.027074       2        1   7.905956
13     0.086722 -0.090521       3        1   6.131061
11     0.054865  0.278779       4        1   5.827622
0      0.036377  0.117494       5        1   5.679532
12    -0.083883 -0.039574       6        1   5.589509
5     -0.014010 -0.129668       7        1   4.989962
8     -0.031539  0.233961       8        1   4.591963
18     0.003230  0.012554       9        1   4.505111
4      0.103106 -0.067735      10        1   4.426145
9      0.065579 -0.023244      11        1   4.417472
10     0.089750 -0.084686      12        1   4.358378
1      0.073334 -0.059652      13        1   3.817308
6      0.046712 -0.019531      14        1   3.798354
19     0.050018 -0.048364      15        1   3.731653
17     0.019913  0.114011      16        1   3.520832
2     -0.038541 -0.096206      17        1   3.490722
7      0.027282 -0.028315      18        1   3.348092
3      0.052053 -0.027577      19        1   2.343599
14     0.064272 -0.006661      20        1   1.909111, topic_info=            Term        Freq       Total Category  logprob  loglift
46         state  393.000000  393.000000  Default  30.0000  30.0000
50          also  357.000000  357.000000  Default  29.0000  29.0000
233         said  678.000000  678.000000  Default  28.0000  28.0000
255         year  237.000000  237.000000  Default  27.0000  27.0000
244   government  232.000000  232.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
2727     startup    5.729460    6.122489  Topic20  -5.3460   3.8922
597       farmer    5.638862    6.031891  Topic20  -5.3619   3.8912
2557       going   28.946537  116.580505  Topic20  -3.7262   2.5654
2724        care    8.192673   22.123583  Topic20  -4.9884   2.9651
2369      reduce    5.817255   12.715215  Topic20  -5.3308   3.1766

[717 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4223     10  0.955717    ability
729      13  0.970586       able
387       4  0.714033  according
387      15  0.292937  according
401      11  0.986195    accused
...     ...       ...        ...
1328     11  0.958504      wrote
255       2  0.996773       year
256       5  0.993378        yet
1239      5  0.974162      young
317      19  0.983419     zverev

[734 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 17, 14, 12, 1, 13, 6, 9, 19, 5, 10, 11, 2, 7, 20, 18, 3, 8, 4, 15])

In [ ]:
pyLDAvis.save_html(vis, 'output_filename.html')

In [ ]:
from IPython.display import Image
Image('pyldavis_consumer.png', height = 800, width = 1200)

#### COMPUTING THE COHERENCE SCORE

In [ ]:
from gensim.models import CoherenceModel
import gensim
import os

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model = lda, texts = word_tokens, dictionary = id2word, coherence = 'c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3341081530614409


#### CREATING THE WORD EMBEDDING

In [ ]:
from gensim.models import word2vec
from sklearn.manifold import TSNE

In [ ]:
#apply the word2vec model using gensim
#the word2vec takes the following arguements
#1. cropus = The vocabulary to train the model on
#2. size = The number of dimensions of the embedding, e.g. the length of the dense vector to represent each token (word).
#3. window = The maximum distance between a target word and words around the target word.
#4. min_count = The minimum count of words to consider when training the model; words with an occurrence less than this count will be ignored.
#5. workers: The number of threads to use while training.
#6. sg: (default 0 or CBOW) The training algorithm, either CBOW (0) or skip gram (1).
model = word2vec.Word2Vec(word_tokens, size = 100, window = 20, min_count = 1000, workers = 2, sg = 0)

RuntimeError: ignored


#### TSNE PLOT

***For better visualizing the TSNE plot, the minimum count of the word while building our word2vec model is taken to be 1000. That means all the words having count less than 1000 will be discarded and our final word2vec model will consist of only those words and their corresponding word vectors which have a count of more than 1000.***

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)

    tsne_model = TSNE(perplexity = 40, n_components = 2, init = 'pca', n_iter = 2500, random_state = 23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])

    plt.figure(figsize = (16, 16))
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy = (x[i], y[i]),
                     xytext = (5, 2),
                     textcoords = 'offset points',
                     ha = 'right',
                     va = 'bottom')
        plt.title('Scatter plot using TSNE')
    plt.show()

In [ ]:
tsne_plot(model)

NameError: ignored

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all')['data']

model = BERTopic("distilbert-base-nli-mean-tokens", verbose=True)
topics = model.fit_transform(docs)


100%|██████████| 245M/245M [00:32<00:00, 7.50MB/s]
2020-10-07 06:45:58,344 - BERTopic - Loaded BERT model
INFO:BERTopic:Loaded BERT model


In [ ]:
!pip install bertopic


     |████████████████████████████████| 4.7MB 3.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 15.9MB/s 
     |████████████████████████████████| 3.0MB 36.4MB/s 
     |████████████████████████████████| 1.1MB 21.0MB/s 
     |████████████████████████████████| 890kB 21.1MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.26-cp36-cp36m-linux_x86_64.whl size=2301860 sha256=98c2e455b38375c24ec349364eefb6207a58a2675f4b2350586cc8d077db72bd
  Stored in directory: /root/.cache/pip/wheels/82/38/41/372f034d8abd271ef7787a681e0a47fc05d472683a7eb088ed
Successfully built hdbscan
  Created wheel for bertopic: filename=bertopic-0.1.2-cp36-none-any.whl size=9614 sha256=c19ef2c2808ba98bddd6bcf288788fa429e4e9007f885bb784977dbaa217086f
  Stored in directory: /root/.cache/pip/wheels/3b/bd/a6/4cea7d9d8edd5d499c7d57ae333b10a

In [ ]:
kaggle competitions download -c 1056lab-defect-detection-extra

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
